In [ ]:
import sqlite3
import pandas as pd

import pystac_client
import odc.stac

In [ ]:
# Connecting to sqlite 
conn = sqlite3.connect('fire.db')

# Create a cursor object
cursor = conn.cursor()

In [ ]:
# set this to True if you run it the first time, to create the database
first = False

if first == True:
    # Read CSV file into a pandas DataFrame
    df = pd.read_csv('./Data/base_data_1986-2018.csv')

    df.drop('time', axis=1, inplace=True)
    df.drop('index_right', axis=1, inplace=True)
    df.month = pd.to_datetime(df.month, format='%Y-%m')

    # Write the DataFrame to the SQLite database
    df.to_sql('ground_data', conn, if_exists='replace', index=False)

    # Commit the changes
    conn.commit()

In [ ]:
cursor.execute('''SELECT * from ground_data LIMIT 10''')
cursor.fetchall()

In [ ]:
#Get all tables
def get_all_tables(cursor):
    sql_query = """SELECT name FROM sqlite_master  
      WHERE type='table';"""
    cursor.execute(sql_query)
    return cursor.fetchall()

# Get column names and data types for each table
def get_table_info(cursor, table_name):
    cursor.execute(f'PRAGMA table_info({table_name})')
    return cursor.fetchall()

In [ ]:
tables = [table[0] for table in get_all_tables(cursor)]

for table in tables:
    print(f"Table: {table}")
    for column in get_table_info(cursor, table):
        print(f"Column: {column[1]}, Type: {column[2]}")
    print("\n")

In [ ]:
catalog = pystac_client.Client.open("https://explorer.dea.ga.gov.au/stac")

In [ ]:
odc.stac.configure_rio(
    cloud_defaults=True,
    aws={"aws_unsigned": True},
)

In [ ]:
def get_month_lat_long(cursor):
    sql_query = '''SELECT strftime('%Y-%m', month) as month, lat, lon from ground_data LIMIT 100'''
    cursor.execute(sql_query)
    return cursor.fetchall()

In [ ]:
data_tiles = get_month_lat_long(cursor)

print(len(data_tiles))

for data in data_tiles:

    bbox = [data[2] - 0.05, data[1] - 0.05, data[2] + 0.05, data[1] + 0.05]

    start_date = f"{data[0][:4]}-01-01"
    end_date = f"{data[0][:4]}-12-31"

    print(f"Searching for data in {start_date} to {end_date} for lat: {data[1]} and long: {data[2]}")

    # Set product ID as the STAC "collection"
    collections = ["ga_ls_fc_pc_cyear_3"]

    # Build a query with the parameters above
    query = catalog.search(
        bbox=bbox,
        collections=collections,
        datetime=f"{start_date}/{end_date}",
    )

    if len(list(query.items())) == 0:
        print("No data found")
        continue

    # Search the STAC catalog for all items matching the query
    items = list(query.items())
    print(f"Found: {len(items):d} datasets")

    # Load the data using the odc.stac module
    ds = odc.stac.load(
                items=items,
                crs="EPSG:3577",
                lat=(bbox[1], bbox[3]),
                lon=(bbox[0], bbox[2]),
                time=(start_date, end_date))

In [ ]:
for item in ds:

    if item == 'qa':
        continue

    mean = ds[item].mean().values

    print(f"Mean value of {item}: {mean}")